# AI4Food Security South Africa

Team EagleEyes:
    
    Christina Bukas, HMGU
    Elisabeth Georgii, HMGU
    Frauke Albrecht, DKRZ
    Caroline Arnold, DKRZ
    

## Introduction

The aim of this challenge is to identify crop types in-season using Planet, Sentinel-1, and Sentinel-2 data. Field boundaries and cropy type labels are given as ground truth data.

## Setup

In the following cell, all modules are imported that are necessary in this notebook.

## Data exploration

### Preprocessing

Data from Planet, Sentinel-1, and Sentinel-2 satellites are given for a region in South Africa. We process the datasets individually, building on the routines that were provided through the starter notebook. The workflow is described here for Planet data.

1. Rasterize the satellite images using the provided routines, for each field save a zip file containing the time series of bands and the mask 
2. Load the field ID and apply custom data transform. Available transforms are: spatial average, crop image, extract fixed number of pixels
3. Save as `hdf5` files for use in training

The full preprocessing module is included in `dataloaders/preprocessor.py`, with the dataset readers in `dataloaders/custom_{planet,sentinel_1,sentinel_2}_reader.py` and the image transforms in `dataloaders/custom_data_transform.py`. 

We generated the training data by running

`python preprocessor.py --region south-africa --data-source planet --t-random-extraction 640 --n-processes 64 --target-sub-dir extracted-640 --overwrite`

`python preprocessor.py --region south-africa --data-source sentinel-1 --t-random-extraction 640 --n-processes 64 --target-sub-dir extracted-640 --overwrite`

`python preprocessor.py --region south-africa --data-source sentinel-2 --t-random-extraction 640 --n-processes 64 --target-sub-dir extracted-640 --overwrite`

This extracts 640 pixels at random from a field, taking into account only the valid pixels as given by the mask. We resample if there are less than 640 pixels are available. Then, out of each field, we create 10 training samples, containing 64 extracted pixels each. Thus, we arrive at a total count of 41430 samples that can be used in training / validation. For the test set, 64 pixels are extracted at random per field.

### Training and validation

We use 10-fold cross validation, dividing the labeled data into training and validation set by field ID.

### Exploration

The crop types are not evenly distributed:

<img src="fig_south_africa_crop_id_barplot.png" alt="bar plot of crop types" title="Available crop types for South Africa" width="350"/>


We calculate the normalized vegetation index NDVI as the normalized difference of infrared and red planet data bands. Below the average NDVI is shown throughout the growing season for the different crop types. 

<img src="fig_south_africa_crop_id_ndvi.png" alt="bar plot of crop types" title="Available crop types for South Africa" width="350"/>


## PSELTAE Model

## Results

<img src="fig_south_africa_predictions_map.png" alt="Predicted crop names for South Africa" title="Predicted crop types for South Africa" width="500"/>
